## Классификация текстов

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

1. Загрузите датасет 20 newsgroups;
2. Познакомьтесь с описанием и структурой датасета. Описание можно найти в документации.
3. Выведите информацию о количественных параметрах датасета;

In [2]:
news = fetch_20newsgroups(subset='all')
print(news.DESCR)

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality

In [3]:
news.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [4]:
print(f'filenames: {news.filenames.shape}')
print(f'target: {news.target.shape}')
print(f'len target_names: {len(news.target_names)}')
print(f'len data: {len(news.data)}')
print(f'target_names: {news.target_names}')

filenames: (18846,)
target: (18846,)
len target_names: 20
len data: 18846
target_names: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


4. Выведите несколько точек датасета (сами текстовые фрагменты и значение целевой переменной);


In [5]:
for i in [5, 25, 55]:
    print(f'------------------\n{news.target_names[news.target[i]]} =>\n------------------\n{news.data[i]}\n\n')

------------------
sci.electronics =>
------------------
From: tell@cs.unc.edu (Stephen Tell)
Subject: Re: subliminal message flashing on TV
Organization: The University of North Carolina at Chapel Hill
Lines: 25
NNTP-Posting-Host: rukbat.cs.unc.edu

In article <7480237@hpfcso.FC.HP.COM> myers@hpfcso.FC.HP.COM (Bob Myers) writes:
>> Hi.  I was doing research on subliminal suggestion for a psychology
>> paper, and I read that one researcher flashed hidden messages on the
>> TV screen at 1/200ths of a second.  Is that possible?

> Might
>even be a vector ("strokewriter") display, in which case the lower limit
>on image time is anyone's guess (and is probably phosphor-persistence limited).

Back in high school I worked as a lab assistant for a bunch of experimental
psychologists at Bell Labs.  When they were doing visual perception and
memory experiments, they used vector-type displays, with 1-millisecond
refresh rates common.

So your case of 1/200th sec is quite practical, and the exper

5. Разделите эти данные на тестовую и обучающую выборки;


In [6]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

6. Постройте модель наивного байесовского для классификации текстов;


In [19]:
vectorizer = CountVectorizer()
vectors_train = vectorizer.fit_transform(newsgroups_train.data)
vectors_test = vectorizer.transform(newsgroups_test.data)

In [24]:
clf = MultinomialNB(alpha=.005)
clf.fit(vectors_train, newsgroups_train.target)

MultinomialNB(alpha=0.005)

In [25]:
pred_train = clf.predict(vectors_train)
metrics.f1_score(newsgroups_train.target, pred_train, average='macro')

0.9704148515443535

In [26]:
pred_test = clf.predict(vectors_test)
metrics.f1_score(newsgroups_test.target, pred_test, average='macro')

0.7843887829880528

7. Оцените качество модели на тестовой выборке с помощью следующих метрик:
   - достоверность предсказания (accuracy);
   - точность (precision);
   - полнота (recall);

8. Постройте кривую обучения - график зависимости тестовой и обучающей эффективности от размера обучающей выборки.

9. Сделайте вывод о применимости модели.

## Дополнительные задания



1. Постройте модели классификации для данной задачи на основе следующих методов:
    - логистическая регрессия (LogisticRegression);
    - метод опорных векторов с гауссовым ядром (SVC);
    - метод опорных векторов с полиномиальным ядром (SVC);
    - метод k ближайших соседей (KNeighborsClassifier);
    - многослойный перцептрон (MLP);
    - другие методы по желанию;


2. Проанализируйте метрики каждой модели и сделайте выводы об их эффективности и применимости. Сравните эффективность всех этих моделей и выберите лучшую;


3. Для каждой модели из п.3 постройте кривые обучения и диагностируйте недо-/переобучение модели. Попробуйте изменить параметр регуляризации для улучшения результатов модели.




4. Сделайте замеры времени обучения для каждой модели. Сделайте вывод о сравнительной производительности моделей.


## Контрольные вопросы



1. Какие выводы мы можем сделать на основании метрик модели, построенной в данной лабораторной работе?


2. Как представляется текст в методах машинного обучения? Какие основные методы векторизации существуют?



3. Какие задачи существую в области обработки естественных текстов?


4. (*) Что такое текстовые вложения (эмбеддинги)? Как они строятся и зачем применяются?

